<a href="https://colab.research.google.com/github/gabrielbrittes/desmatamento/blob/main/desmatamento_brasil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import necessário do kaggle
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [ ]:
#Download do Dataset
!kaggle datasets download -d fidelissauro/desmatamento-brasil
!chmod 600 /content/kaggle.json

  0% 0.00/1.07k [00:00<?, ?B/s]
100% 1.07k/1.07k [00:00<00:00, 2.78MB/s]


In [ ]:
#Descompactando o arquivo, não precisará caso o arquivo já esteja aqui (o deixei no drive)
!unzip \*.zip && rm *.zip

Archive:  desmatamento-brasil.zip
  inflating: desmatamento_prodes.csv  


In [ ]:
#Verificando o processamento dos dados com o pyspark
!pip install pyspark

# Cria a SparkSession
from pyspark.sql import SparkSession

# Cria uma nova instância do SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

# Lê o CSV para um dataframe, utilizando header e partindo do ponto que o CSV é separado por virgulas
dados = (
    spark
    .read
    .csv("/content/desmatamento_prodes.csv", header=True, sep=",", inferSchema=True)
)

# Printa o schema do DataFrame
dados.printSchema()

# Mostra os dados do DataFrame
dados.show()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=04110bbf6d225efe3ae64496cbbda6e5b92c0944d0be6ef99a61991a1c74e043
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
root
 |-- referencia: integer (nullable = true)
 |-- acre: integer (nullable = true)
 |-- amazonas: integer (nullable = true)
 |-- amapa: integer (nullable = true)
 |-- maranhao: integer (nullable = true)
 |-- mato_grosso: integer (nullable = true)
 |-- para: integer (nullable = true)
 |-- rondonia: integer (nullable = true)
 |-- roraima: integer (nullable = true)
 |-- tocantins: integer (nullable = true)
 |-- area_total_desmatamento: integer (nullable = true)

+----------+----+--------+-----+--------+-----------+----+--------+-------+---------+-----------------------+
|referen

In [ ]:
#Criando uma view para testar os dados
dados.createOrReplaceTempView('view_dados_temp')

In [ ]:
# Testando se os dados foram 'ingeridos' e estão manipulaveis
# area_total_desmatamentos: número de novos óbitos desde o último dia, estarei ordenando pelo maior valor
spark.sql("""
    SELECT *
      FROM view_dados_temp
     ORDER BY area_total_desmatamento DESC
""").show()

+----------+----+--------+-----+--------+-----------+----+--------+-------+---------+-----------------------+
|referencia|acre|amazonas|amapa|maranhao|mato_grosso|para|rondonia|roraima|tocantins|area_total_desmatamento|
+----------+----+--------+-----+--------+-----------+----+--------+-------+---------+-----------------------+
|      1995|1208|    2114|    9|    1745|      10391|7845|    4730|    220|      797|                  29059|
|      2004| 728|    1232|   46|     755|      11814|8870|    3858|    311|      158|                  27772|
|      2003|1078|    1558|   25|     993|      10405|7145|    3597|    439|      156|                  25396|
|      2002| 883|     885|    0|    1085|       7892|7510|    3099|     84|      212|                  21650|
|      1988| 620|    1510|   60|    2450|       5140|6990|    2340|    290|     1650|                  21050|
|      2005| 592|     775|   33|     922|       7145|5899|    3244|    133|      271|                  19014|
|      200